In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np

print("Gerando amostras de saldos PF e PJ...")

# Parâmetros
pf_mean = 6.0
pf_sigma = 1.5
pj_mean = 9.0
pj_sigma = 1.8
tamanho_amostra = 100000

# Dados sintéticos
saldos_pf = np.random.lognormal(mean=pf_mean, sigma=pf_sigma, size=tamanho_amostra)
saldos_pj = np.random.lognormal(mean=pj_mean, sigma=pj_sigma, size=tamanho_amostra)

# DataFrames
df_pf = pd.DataFrame({'Saldo': saldos_pf, 'Tipo': 'Pessoa Física (PF)'})
df_pj = pd.DataFrame({'Saldo': saldos_pj, 'Tipo': 'Pessoa Jurídica (PJ)'})
df_saldos_plot = pd.concat([df_pf, df_pj], ignore_index=True)

print("Gerando o gráfico comparativo aprimorado...")

# Figura e eixo
fig, ax = plt.subplots(figsize=(15, 8))

# Gráfico KDE
sns.kdeplot(
    data=df_saldos_plot,
    x='Saldo',
    hue='Tipo',
    fill=True,
    common_norm=False,
    alpha=0.5,
    linewidth=3,
    log_scale=True,
    ax=ax,
    legend=False  # Desativa a legenda automática
)

# Títulos e rótulos
ax.set_title('Comparação da Distribuição de Saldos Gerados: PF vs. PJ', fontsize=18)
ax.set_xlabel('Saldo da Conta (em R$, Escala Logarítmica)', fontsize=13)
ax.set_ylabel('Densidade Estimada', fontsize=13)

# Eixo X logarítmico formatado
major_ticks = [
    10 ** i
    for i in range(
        int(np.log10(df_saldos_plot['Saldo'].min())),
        int(np.log10(df_saldos_plot['Saldo'].max())) + 2
    )
]
ax.set_xticks(major_ticks)
formatter = FuncFormatter(lambda x, _: f'R$ {x:,.0f}')
ax.xaxis.set_major_formatter(formatter)
plt.setp(ax.get_xticklabels(), rotation=30, ha='right', fontsize=11)

# ✅ Recriar legenda manualmente
handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles,
    labels,
    title='Tipo de Cliente',
    fontsize=12,
    title_fontsize=13,
    loc='upper right'
)

# Grid e layout
ax.grid(True, which="both", ls="--", linewidth=0.5)
plt.tight_layout()

plt.show()


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter, LogLocator
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# --- 1. Parâmetros de Análise (Ajuste conforme necessário) ---
ano_alvo = 2023 # Ano da sua simulação
mes_alvo = 1 # Escolha o mês que você quer analisar (1 a 12)
# Use o ID do município que você quer analisar (ex: o Top 1 que encontramos antes)
# Se a variável top_municipio_id não existir mais, substitua pelo número, ex: 3550308
try:
    municipio_alvo_id = top_municipio_id 
    municipio_alvo_nome = top_municipio_nome # Para o título
except NameError:
    # Se as variáveis não existirem, defina manualmente (exemplo para São Paulo)
    municipio_alvo_id = 3550308 
    municipio_alvo_nome = "Município Selecionado" 
    print(f"AVISO: Usando município de exemplo ID: {municipio_alvo_id}. Ajuste se necessário.")
    
tabela_sintetica_transacoes = "transacoes_db.copper.transacoes"
tabela_sintetica_contas = "transacoes_db.copper.contas"
tamanho_amostra_plot = 200000 # Amostra para o gráfico (pode aumentar se precisar de mais detalhe)

print(f"Iniciando análise da distribuição de valores para {municipio_alvo_nome}, Mês: {mes_alvo}/{ano_alvo}...")

# --- 2. SQL para Buscar os Valores das Transações Sintéticas ---
sql_query_valores = f"""
SELECT
    t.valor
FROM {tabela_sintetica_transacoes} AS t
JOIN {tabela_sintetica_contas} AS c ON t.id_conta_origem = c.id
WHERE
    c.municipio_ibge = {municipio_alvo_id}
    AND YEAR(t.data) = {ano_alvo}
    AND MONTH(t.data) = {mes_alvo}
"""

print("Executando query SQL para buscar valores das transações sintéticas...")
df_valores_spark = spark.sql(sql_query_valores)

# --- 3. Amostragem e Conversão para Pandas ---
total_registros_mes = df_valores_spark.count()

if total_registros_mes == 0:
    print(f"AVISO: Nenhuma transação sintética encontrada para o Mês {mes_alvo}/{ano_alvo} no município {municipio_alvo_id}.")
else:
    fracao = min(1.0, tamanho_amostra_plot / total_registros_mes)
    
    print(f"Coletando uma amostra aleatória de aproximadamente {int(total_registros_mes * fracao)} registros para plotagem...")
    # Usa cache para acelerar a amostragem se df_valores_spark for grande
    df_valores_sample = df_valores_spark.sample(withReplacement=False, fraction=fracao, seed=42)
    valores_pd = df_valores_sample.toPandas() # Libera o cache

    # --- 4. Gerar o Gráfico de Distribuição ---
    print("Gerando o gráfico de distribuição de valores...")
    plt.figure(figsize=(14, 8))

    # Usar histplot para mostrar barras e curva, com escala logarítmica
    sns.histplot(data=valores_pd, x='valor', 
                 bins=100, kde=True, 
                 line_kws={'linewidth': 3, 'color': 'red'},
                 log_scale=True) # Escala Log no eixo X

    plt.title(f'Distribuição dos Valores Finais ($V_{{tx}}$) das Transações - {municipio_alvo_nome} (Mês {mes_alvo}/{ano_alvo})', fontsize=16)
    plt.xlabel('Valor da Transação (em R$, Escala Logarítmica)', fontsize=12)
    plt.ylabel('Frequência / Densidade Estimada', fontsize=12)

    # Formatar eixo X
    formatter = FuncFormatter(lambda x, _: f'R$ {x:,.0f}' if x >= 1 else f'R$ {x:.2f}') 
    plt.gca().xaxis.set_major_formatter(formatter)
    
    # Definir ticks principais para escala log
    min_val = max(0.01, valores_pd['valor'].min()) # Evita log(0)
    max_val = valores_pd['valor'].max()
    major_ticks = [10**i for i in range(int(np.floor(np.log10(min_val))), int(np.ceil(np.log10(max_val))) + 1)]
    plt.gca().set_xticks(major_ticks)
    plt.xticks(rotation=30, ha='right')

    plt.grid(True, which="both", ls="--", linewidth=0.5)
    plt.tight_layout()
    plt.show()
    
    # Informações adicionais (opcional)
    print("\nEstatísticas Descritivas dos Valores Plotados:")
    print(valores_pd['valor'].describe().apply(lambda x: f"{x:,.2f}"))